# Démo JobSwipe Generator

Ce notebook démontre l'utilisation des différentes fonctions de génération (Parsing, Scoring, Réécriture CV, Lettre de motivation) utilisant Gemini.

In [1]:
import os
import sys
import json
try:
    from dotenv import load_dotenv
except ImportError:
    raise ImportError("Erreur d'import : installez 'python-dotenv' (et non 'dotenv'). Commande : pip uninstall dotenv; pip install python-dotenv")

load_dotenv()

# Si vous n'avez pas de fichier .env, décommentez et mettez votre clé ici :
# os.environ["GEMINI_API_KEY"] = "VOTRE_CLE_ICI"

# Ajout du dossier courant au path pour les imports si nécessaire
sys.path.append(os.getcwd())


## 1. Import des fonctions

In [2]:
from cv_parsing import parse_cv_with_gemini
from job_offer_parser import parse_job_offer_gemini
from compatibility import score_profile_with_gemini
from experience_generator import generate_full_cv_content
from cv_generator import generate_cv_html, convert_html_to_pdf
from cover_letter_generator import generate_personalized_cover_letter_docx_and_pdf

c:\Users\theau\OneDrive\Documents\IMT\3A\PROCOM\MVP\jobswipe-career-navigator\backend\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


model name gemini-3-flash-preview


## 2. Définition des entrées (CV brut et Offre d'emploi)

In [3]:
raw_cv_text = """
THEAU AGUET
Data Scientist & Entrepreneur
France
Phone: +33 7 82 01 83 97
Email: theauaguetpro@gmail.com

EXPERIENCE
Data Scientist Intern - Airbus, Toulouse
Feb 2024 - Aug 2024
- Built predictive maintenance models on sensor data (Python, scikit-learn).
- Improved failure detection by ~7% on 50k+ records.
- Automated dashboards for 3 stakeholders.

EDUCATION
IMT Atlantique – Diplôme d'ingénieur en Data Science
2022 – 2025

SKILLS
Python, SQL, Machine Learning, D3.js, Docker
"""

raw_job_offer = """
Data Scientist Junior (H/F) - Airbus Defence and Space
Toulouse, France

Missions :
- Analyser des données capteurs pour la maintenance prédictive.
- Développer des modèles de machine learning en Python.
- Mettre en production les modèles via Docker.

Profil :
- Bac+5 en Data Science.
- Maîtrise de Python, SQL et des bibliothèques ML.
- Bon niveau d'anglais.
"""

## 3. Parsing par l'IA

In [4]:
print("Parsing du CV...")
cv_parsed = parse_cv_with_gemini(raw_cv_text)
print("CV Parsé (clés) :", cv_parsed.keys())

print("\nParsing de l'offre...")
offer_parsed = parse_job_offer_gemini(raw_job_offer)
print("Offre Parsée (titre) :", offer_parsed.get("title"))

Parsing du CV...
CV Parsé (clés) : dict_keys(['first_name', 'last_name', 'full_name', 'contacts', 'websites', 'social_links', 'skills', 'professional_experiences', 'academic_projects', 'education', 'certifications', 'interests', 'raw_summary'])

Parsing de l'offre...
Offre Parsée (titre) : Data Scientist Junior (H/F)


## 4. Calcul du score de compatibilité

In [5]:
compatibility = score_profile_with_gemini(offer_parsed, cv_parsed)
print(json.dumps(compatibility, indent=2, ensure_ascii=False))

{
  "overall_score": 96,
  "scores": {
    "skills_match": 95,
    "experience_match": 98,
    "education_match": 100,
    "language_match": 85
  },
  "summary": "Theau is an ideal candidate who has already completed a 6-month internship at Airbus in the exact field of predictive maintenance. His technical profile perfectly matches the requirements, and his educational background from a top-tier engineering school (IMT Atlantique) aligns with the 'Bac+5' requirement.",
  "key_strengths": [
    "Direct previous experience at Airbus in the specific domain of predictive maintenance.",
    "Full alignment with the required tech stack, including Python, SQL, and Docker.",
    "Strong academic background with a specialized engineering degree in Data Science.",
    "Proven impact during internship, improving failure detection by 7%."
  ],
  "key_gaps": [
    "Language proficiency levels (English/French) are not explicitly stated in the skills section.",
    "Limited professional experience ou

## 5. Génération du contenu du CV sur-mesure

In [9]:
# Préparation des données pour le générateur unique
user_data = {
    "profile": {
        "summary": cv_parsed.get("raw_summary")
    },
    "experiences": cv_parsed.get("professional_experiences", []),
    "projects": cv_parsed.get("academic_projects", []),
    "education": cv_parsed.get("education", []),
    "skills": cv_parsed.get("skills", {}),
    "interests": cv_parsed.get("interests", [])
}

print("Génération du CV complet (One-Shot)...")
generated_content = generate_full_cv_content(offer_parsed, user_data)

# Adaptation du format pour cv_generator (qui attend des dicts imbriqués)
full_cv_content = {
    "cv_title": {"cv_title": generated_content.get("cv_title")},
    "objective": {"objective": generated_content.get("objective")},
    "experiences": {"experiences": generated_content.get("experiences", [])},
    "projects": {"projects": generated_content.get("projects", [])},
    "education": {"education": generated_content.get("education", [])},
    "skills": {"skills": generated_content.get("skills", {})},
    "interests": {"interests": generated_content.get("interests", [])}
}

print("Contenu généré avec succès.")

Sélection du contenu pertinent...
[WARN] Erreur lors de la sélection des expériences : 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-3-flash
Please retry in 58.992824567s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-3-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 20
}
, retry_delay {
  seconds: 58
}
]
[WARN] Erreur lo

ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-3-flash
Please retry in 58.867408281s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-3-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 20
}
, retry_delay {
  seconds: 58
}
]

## 6. Création du PDF du CV (HTML -> PDF)

In [10]:
# Extraction des infos de contact depuis le parsing initial
contacts = cv_parsed.get("contacts") or {}
contact_info = {
    "name": cv_parsed.get("full_name") or "Candidat",
    "city": (contacts.get("locations") or [""])[0],
    "phone": (contacts.get("phones") or [""])[0],
    "email": (contacts.get("emails") or [""])[0],
    "linkedin": "linkedin.com/in/demo",
    "github": "github.com/demo",
    "role": full_cv_content.get("cv_title", {}).get("cv_title", "Data Scientist")
}

output_dir = "output_demo"
os.makedirs(output_dir, exist_ok=True)

print("Génération du HTML...")
html_content = generate_cv_html(full_cv_content, contact_info)
html_path = os.path.join(output_dir, "resume.html")
with open(html_path, "w", encoding="utf-8") as f:
    f.write(html_content)

print("Conversion en PDF...")
pdf_path = os.path.join(output_dir, "resume.pdf")
convert_html_to_pdf(html_content, pdf_path)
print(f"CV généré : {pdf_path}")

Génération du HTML...


NameError: name 'full_cv_content' is not defined

## 7. Génération de la Lettre de Motivation

In [ ]:
print("Génération de la lettre de motivation...")
cl_paths = generate_personalized_cover_letter_docx_and_pdf(
    offer_parsed=offer_parsed,
    cv_parsed=cv_parsed,
    output_dir=output_dir,
    docx_filename="lettre_motivation_demo.docx",
    gender="M"
)
print(f"Lettre DOCX : {cl_paths['docx_path']}")
print(f"Lettre PDF : {cl_paths['pdf_path']}")

Génération de la lettre de motivation...


PermissionError: [Errno 13] Permission denied: 'c:\\Users\\theau\\OneDrive\\Documents\\IMT\\3A\\PROCOM\\MVP\\jobswipe-career-navigator\\backend\\generator\\functions\\output_demo\\lettre_motivation_demo.docx'